In [2]:
import pandas as pd

tikidf = pd.read_csv(
            'D:/Desktop/Nivea-Assignment/tiki-scrape/data.csv',
            encoding_errors='replace')

lazadf = pd.read_csv(
            'D:/Desktop/Nivea-Assignment/lazada-scrape/data.csv',
            encoding_errors='replace')

df = pd.concat([tikidf, lazadf], ignore_index=True)

In [3]:
niveamen_mask1 = df['name'].str.contains('nivea', case=False, na=False)
niveamen_mask2 = df['name'].str.contains('|'.join(['men', 'nam']), case=False, na=False)
niveamen_products = df[niveamen_mask1 & niveamen_mask2]
niveamen_products.reset_index(drop=True)
print (niveamen_products)

     platform          id                                               name  \
0        tiki   273455559  lan-ngan-mui-nivea-men-silver-protect-phan-tu-...   
1        tiki   273455553  lan-ngan-mui-nivea-men-dry-impact-kho-thoang-5...   
2        tiki    54414235  xit-ngan-mui-nivea-men-silver-protect-phan-tu-...   
3        tiki   273455567  lan-ngan-mui-nivea-men-black-white-ngan-vet-o-...   
4        tiki    54384691  xit-ngan-mui-nivea-men-dry-impact-kho-thoang-1...   
...       ...         ...                                                ...   
1933   lazada  2594644910  mua-1-tang-1-sua-tam-goi-danh-cho-nam-nivea-me...   
1935   lazada  2422021320  bo-doi-lan-ngan-mui-50ml-va-xit-ngan-mui-150ml...   
1937   lazada  2067001646  sua-rua-mat-nam-nivea-cho-da-nhon-extra-white-...   
1945   lazada  2478595079  bo-4-sap-khu-mui-nivea-men-phan-tu-bac-50-ml-8...   
1947   lazada  2687866631  sua-tam-goi-xa-nivea-men-3in1-sensitive-soothi...   

        sale     price                 

In [4]:
from pydantic import BaseModel

class RawProduct(BaseModel):
    platform : str
    id: int | None
    name: str | None
    sale: int | None
    price: float | None
    shop_name: str | None
    shop_id: int | None
    unite_id: int = None
    brand_name: str | None
    timestamp: float | None
    
class TransformedShop(BaseModel):
    platform : str
    name : str
    shop_revenue: float
    shop_unitsold: int

class TransformedProduct(BaseModel):
    platform : str
    name : str
    shop_name : str
    price : float
    unit_sold : int
    revenue : float
    
class AggregationResult(BaseModel):
    related_product_names_quantity : int
    related_product_unit_solds : int
    related_product_revenue : float
    related_shop_quantity : int

class AggregationPlatformResult(AggregationResult):
    platform : str

In [5]:
from typing import List

# Step 1: Generate TransformedShop Data
def generate_transformed_shops(df: pd.DataFrame) -> List[TransformedShop]:
    if 'revenue' not in df.columns: df['revenue'] = df['sale'] * df['price']  # Calculate revenue
    shop_data = (
        df.groupby(['platform', 'shop_name'])
        .agg(shop_revenue=('revenue', 'sum'), shop_unitsold=('sale', 'sum'))
        .reset_index()
    )
    shop_data = shop_data.rename(columns={'shop_name':'name'})
    return [TransformedShop(**row) for _, row in shop_data.iterrows()]

# Step 2: Generate TransformedProduct Data
def generate_transformed_products(df: pd.DataFrame) -> List[TransformedProduct]:
    if 'revenue' not in df.columns: df['revenue'] = df['sale'] * df['price']  # Calculate revenue
    product_data = df[['platform', 'name', 'shop_name', 'price', 'sale', 'revenue']]
    product_data = product_data.rename(columns={'sale': 'unit_sold'})
    return [TransformedProduct(**row) for _, row in product_data.iterrows()]

# Step 3: Generate AggregationResult Data
def generate_aggregation_result(df: pd.DataFrame) -> AggregationResult:
    if 'revenue' not in df.columns: df['revenue'] = df['sale'] * df['price']  # Calculate revenue
    related_product_names_quantity = df['name'].nunique()
    related_product_unit_solds = df['sale'].sum()
    related_product_revenue = df['revenue'].sum()
    related_shop_quantity = df['shop_name'].nunique()
    return AggregationResult(
        related_product_names_quantity=related_product_names_quantity,
        related_product_unit_solds=related_product_unit_solds,
        related_product_revenue=related_product_revenue,
        related_shop_quantity=related_shop_quantity
    )

# Step 4: Generate AggregationPlatformResult Data
def generate_aggregation_platform_results(df: pd.DataFrame) -> List[AggregationPlatformResult]:
    if 'revenue' not in df.columns: df['revenue'] = df['sale'] * df['price']  # Calculate revenue
    platform_agg_data = (
        df.groupby('platform')
        .agg(
            related_product_names_quantity=('name', 'nunique'),
            related_product_unit_solds=('sale', 'sum'),
            related_product_revenue=('revenue', 'sum'),
            related_shop_quantity=('shop_name', 'nunique')
        )
        .reset_index()
    )
    return [AggregationPlatformResult(**row) for _, row in platform_agg_data.iterrows()]

# Example of processing
# Assuming `raw_products_df` is the pandas DataFrame with `RawProduct` structure
raw_products_df = niveamen_products

# Calculate TransformedShop, TransformedProduct, AggregationResult, AggregationPlatformResult
transformed_shops = generate_transformed_shops(raw_products_df)
transformed_products = generate_transformed_products(raw_products_df)
aggregation_result = generate_aggregation_result(raw_products_df)
aggregation_platform_results = generate_aggregation_platform_results(raw_products_df)

C:\Users\Admin\AppData\Local\Temp\ipykernel_5788\2325796340.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if 'revenue' not in df.columns: df['revenue'] = df['sale'] * df['price']  # Calculate revenue


#### Dashboard Demand Dataformat

{
    'tiki': {
        'revenue': <money-str>, 
        'unitSold': <bigint-str>, 
        'numProduct': <bigint-str>,
        'numShop': <bigint-str>,
    },
    'lazada': {
        'revenue': <money-str>, 
        'unitSold': <bigint-str>, 
        'numProduct': <bigint-str>,
        'numShop': <bigint-str>,
    }
    'total': {
        'revenue': <money-str>, 
        'unitSold': <bigint-str>, 
        'numProduct': <bigint-str>,
        'numShop': <bigint-str>,
    },
    'priceDist': {
        'labels': [<ranges-of-price>],
        'data': [<unitSold-by-price-range>],
    }
}

* <money-str> : str of float with a $ at front with , delimeter for each 3 digits
* <bigint-str> : str of int with , delimeter for each 3 digits

In [9]:
import pandas as pd
import numpy as np
import json

MONEY_SIGN = "VND "
# Step 5: Calculate Price Distribution
def calculate_price_distribution(df: pd.DataFrame) -> dict:
    prices = df['price'].dropna()  # Assuming 'price' is a column in your DataFrame
    histogram, bin_edges = np.histogram(prices, bins='auto')  # Automatic binning

    # Create labels for bins
    price_labels = [f"{MONEY_SIGN}{bin_edges[i]:.2f} - {MONEY_SIGN}{bin_edges[i+1]:.2f}" for i in range(len(bin_edges) - 1)]
    
    return {
        'labels': price_labels,
        'data': histogram.tolist()  # Convert numpy array to list for JSON serialization
    }

# Step 6: Generate Dashboard Data
def generate_dashboard_data(raw_products_df: pd.DataFrame) -> dict:
    # if transformed_shops is None:
    transformed_shops = generate_transformed_shops(raw_products_df) 
    # if transformed_products is None:
    transformed_products = generate_transformed_products(raw_products_df) 
    # if aggregation_result is None:
    aggregation_result = generate_aggregation_result(raw_products_df) 
    # if aggregation_platform_results is None:
    aggregation_platform_results = generate_aggregation_platform_results(raw_products_df) 

    # Revenue and unit sold calculations
    dashboard_data = {}
    total_revenue = 0
    total_units_sold = 0
    total_num_products = len(raw_products_df['name'].unique())
    total_num_shops = len(raw_products_df['shop_name'].unique())

    for platform in raw_products_df['platform'].unique():
        platform_data = [shop for shop in transformed_shops if shop.platform == platform]
        revenue = sum(shop.shop_revenue for shop in platform_data)
        units_sold = sum(shop.shop_unitsold for shop in platform_data)
        numProduct = len([product for product in transformed_products if product.platform == platform])

        dashboard_data[platform] = {
            'revenue': f"{MONEY_SIGN}{revenue:,.2f}",
            'unitSold': f"{units_sold:,}",
            'numProduct': f"{numProduct:,}",
            'numShop': f"{len(platform_data):,}"  # Unique shops for the platform
        }

        total_revenue += revenue
        total_units_sold += units_sold

    # Total summary
    dashboard_data['total'] = {
        'revenue': f"{MONEY_SIGN}{total_revenue:,.2f}",
        'unitSold': f"{total_units_sold:,}",
        'numProduct': f"{total_num_products:,}",
        'numShop': f"{total_num_shops:,}",
    }
    
    # Price Distribution
    dashboard_data['priceDist'] = calculate_price_distribution(raw_products_df)

    return dashboard_data

# Example of processing
# Assuming `raw_products_df` is the pandas DataFrame with `RawProduct` structure
raw_products_df = niveamen_products

# Generate Dashboard Data
dashboard_json = generate_dashboard_data(raw_products_df)
with open('niveamen_sale.json',mode='w') as file:
    file.write(json.dumps(dashboard_json, indent=4))